In [15]:
import pandas as pd
import numpy as np
import importlib
import utilities.utility as utility
from pypfopt import EfficientFrontier, risk_models, expected_returns

import utilities.variables as variables
import utilities.api_ticker_service as data_ticker_service
importlib.reload(data_ticker_service)
importlib.reload(variables)

<module 'utilities.variables' from '/Users/herbishtini/Documents/UNI/Master Thesis/sustainability_portfolio_optimisation/utilities/variables.py'>

### Average historical return

In [27]:
df_monthly_adj_close = pd.read_csv('../data/10_monthly_adjacent_close.csv', index_col=0)

#### Converting 'Adj Close' Values to Percentage Change
The "Adj Close" will be transformed to represent the percentage change from the previous day's close. The percentage change will be calculated as 1 + percentage change, which indicates the relative change in the adjusted close values day-over-day.

In [28]:
df_monthly_return = df_monthly_adj_close.copy(deep=True)
for column in df_monthly_return:
    if column != 'Date':
        df_monthly_return[column] = 1 + df_monthly_adj_close[column].pct_change().round(2)
        df_monthly_return[column].dropna()

In [18]:
df_monthly_return.to_csv('../data/10_monthly_returns_complete.csv')

### Return rate

In [29]:
df_monthly_return = pd.read_csv('../data/10_monthly_returns_complete.csv', index_col=0)
df_overview = pd.read_csv('../data/data_5_scaled.csv', index_col=0)

In [34]:
df_monthly_return.head()

,Date,KE,SCS,HNI,AVT,ACCO,KEYS,CBRE,BRC,PGRE,...,ENPH,RUN,SUP,MATX,KIDS,HALO,MATW,KVHI,NEO,UNP
0,1999-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1999-09-01,NaN,0.94,0.82,0.95,NaN,NaN,NaN,1.07,NaN,...,NaN,NaN,1.00,1.00,NaN,NaN,1.11,0.97,NaN,0.99
2,1999-10-01,NaN,0.91,1.02,1.30,NaN,NaN,NaN,0.91,NaN,...,NaN,NaN,0.95,1.01,NaN,NaN,0.83,1.29,NaN,1.16
3,1999-11-01,NaN,1.03,1.10,1.01,NaN,NaN,NaN,1.06,NaN,...,NaN,NaN,1.01,0.95,NaN,NaN,0.93,0.99,NaN,0.84
4,1999-12-01,NaN,0.92,1.02,1.10,NaN,NaN,NaN,1.11,NaN,...,NaN,NaN,1.00,1.01,NaN,NaN,1.18,1.01,NaN,0.93


In [32]:
# move Date index to a new column
df_monthly_adj_close = df_monthly_adj_close.reset_index(level=0)
df_monthly_return = df_monthly_return.reset_index(level=0)

# Creates a new column for each year group (1, 5, 10, 25 year returns)
utility.set_yearly_return_rates_by_years(df_overview, df_monthly_return);

In [33]:
df_overview

,company_name,company_esg_score,company_esg_score_group,industry,stock_exchange,stock_ticker_symbol,market_capital,market_capital_euro,trailing_pe,beta,return_on_equity,earnings_growth,company_esg_score_scale,market_capital_scale,score,return_rate_1y_avg,return_rate_5y_avg,return_rate_10y_avg,return_rate_25y_avg
0,RS Group Plc,4.5,Negligible ESG Risk,Technology Hardware,LON,RS1.L,3.668791e+09,4.365861e+09,19.858974,0.863,0.13226,-0.368,0.000000,0.000273,0.000205,NaN,NaN,NaN,NaN
1,"Kimball Electronics, Inc.",4.5,Negligible ESG Risk,Technology Hardware,NAS,KE,4.281351e+08,3.938843e+08,21.370369,1.258,0.03854,NaN,0.000000,0.000024,0.000018,0.881384,1.114518,1.077999,1.030499
2,TAG Immobilien AG,4.6,Negligible ESG Risk,Real Estate,ETR,TEG.DE,2.765612e+09,2.765612e+09,NaN,1.037,-0.03766,NaN,0.006452,0.000173,0.001743,NaN,NaN,NaN,NaN
3,LEG Immobilien SE,5.1,Negligible ESG Risk,Real Estate,ETR,LEG.DE,6.927171e+09,6.927171e+09,NaN,0.957,-0.08126,NaN,0.038710,0.000434,0.010003,NaN,NaN,NaN,NaN
4,"Steelcase, Inc.",5.3,Negligible ESG Risk,Commercial Services,NYS,SCS,1.492860e+09,1.373431e+09,17.210526,1.314,0.10633,8.000,0.051613,0.000086,0.012968,1.299105,0.991720,1.025479,1.036678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,"KVH Industries, Inc. (Delaware)",20.0,Low ESG Risk,Technology Hardware,NAS,KVHI,9.127629e+07,9.127629e+07,NaN,0.594,-0.14536,NaN,1.000000,0.000006,0.250004,0.938140,0.850474,0.917683,1.030659
1658,Moonpig Group Plc,20.0,Low ESG Risk,Retailing,LON,MOON.L,7.018797e+08,7.018797e+08,2035.000000,1.238,NaN,-0.072,1.000000,0.000044,0.250033,NaN,NaN,NaN,NaN
1659,"NeoGenomics, Inc.",20.0,Low ESG Risk,Pharmaceuticals,NAS,NEO,1.940312e+09,1.940312e+09,NaN,1.191,-0.08391,NaN,1.000000,0.000121,0.250091,1.212992,0.950939,1.113462,1.156116
1660,"Japan Material Co., Ltd.",20.0,Low ESG Risk,Semiconductors,TKS,6055.T,1.704423e+11,1.704423e+11,27.933996,0.585,0.13585,0.379,1.000000,0.010681,0.258011,NaN,NaN,NaN,NaN


### Volatility
To calculate the volatility of a stock's monthly returns, there are several steps that should be applied to the historical price data.
* Calculate Monthly Returns
* Calculate the Average Monthly Return
* Calculate the Monthly Return Deviations
* Square the Deviations
* Calculate the Variance
* Calculate the Standard Deviation

In [22]:
utility.set_volatility_by_years(df_overview, df_monthly_adj_close)

In [24]:
df_overview[['stock_ticker_symbol', 'return_rate_5y_avg', 'return_rate_10y_avg', 'return_rate_25y_avg', 'volatility_10y']].sort_values(by=['return_rate_10y_avg', 'volatility_10y'])

,stock_ticker_symbol,return_rate_5y_avg,return_rate_10y_avg,return_rate_25y_avg,volatility_10y
442,FOSL,0.625339,0.647001,0.913120,0.239848
1453,GPRO,0.787331,0.663986,0.871982,0.164852
1235,AMC,0.547253,0.702343,0.871146,0.568230
1594,BIG,0.547151,0.706566,0.893492,0.193526
897,WW,0.494756,0.727525,0.877270,0.281869
...,...,...,...,...,...
1651,9260.T,NaN,NaN,NaN,NaN
1652,DEQ.DE,NaN,NaN,NaN,NaN
1656,9842.T,NaN,NaN,NaN,NaN
1658,MOON.L,NaN,NaN,NaN,NaN


### Store changes

In [26]:
df_overview.to_csv('../data/data_10_overview.csv')
df_monthly_return.to_csv('../data/data_10_monthly_returns_complete.csv')